# Transform Degrees in Latitude to Distance in km

In order to retrieve equally sized images from across the globe, the variation of the relation between distance and degrees (lon and lat) must be considered.

In [2]:
from geopy.distance import geodesic
from geopy.point import Point
import numpy as np

# Latitude to distance and back

In [3]:
span = 0.045022463190788
center = Point(4.093329286999995, -74.91028175232206)
center_north = Point(4.093329286999995 - (span / 2), -74.91028175232206)
center_south = Point(4.093329286999995 + (span / 2), -74.91028175232206)

distance = geodesic(center_north, center_south).km

destination_north = geodesic(kilometers=(distance / 2)).destination(center, 180)
destination_south = geodesic(kilometers=(distance / 2)).destination(center, 0)

# Both results are amlost identical, as expected.
print(center_north, center_south)
print(destination_north, destination_south)

4 4m 14.945s N, 74 54m 37.0143s W 4 6m 57.0259s N, 74 54m 37.0143s W
4 4m 14.945s N, 74 54m 37.0143s W 4 6m 57.0258s N, 74 54m 37.0143s W


In [4]:
def compute_img_span_km(img_center_lon: float, img_center_lat: float, img_span_lat: float) -> float:
    """Compute the distance in kilometer of the image span in latitude direction.

    Args:
        img_center_lon (float): Longitude of the image center.
        img_center_lat (float): Latitude of the image center. 
        img_span_lat (float): Span of the image (north to south) in degrees latitude.  

    Returns:
        float: Span of the image in kilometer. 
    """
    # Offset is half the image span. Turn offset negative on the southern hemisphere
    lat_offset = (img_span_lat / 2) if img_center_lat > 0 else -(img_span_lat / 2)
    img_northern = Point(img_center_lat + lat_offset, img_center_lon) 
    img_southern = Point(img_center_lat - lat_offset, img_center_lon)
    return geodesic(img_northern, img_southern).km

In [5]:
def compute_img_span_lat(img_center_lon: float, img_center_lat: float, img_span_km: float) -> float:
    """Compute the distance in degrees latitude from kilometers.

    Args:
        img_center_lon (float): Longitude of the image center.
        img_center_lat (float): Latitude of the image center. 
        img_span_km (float): Span of the image in kilometer.  

    Returns:
        float: Span of the image (north to south) in degrees latitude.
    """
    img_center = Point(img_center_lat, img_center_lon)
    img_northern = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 180)
    img_southern = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 0)
    return np.abs(img_northern.latitude - img_southern.latitude)

In [6]:
img_center_lat, img_center_lon = 4.093329286999995, -74.91028175232206
img_span_lat = 0.045022463190788

img_span_km = compute_img_span_km(img_center_lon, img_center_lat, img_span_lat)
img_span_lat_rev = compute_img_span_lat(img_center_lon, img_center_lat, img_span_km)

print(img_span_km, img_span_lat, img_span_lat_rev)

4.978580991171369 0.045022463190788 0.04502246319079539


# Distance to Latitude and Longitude

In [7]:
def compute_img_span(img_center_lon: float, img_center_lat: float, img_span_km: float) -> tuple[float, float]:
    """Compute the distance in degrees longitude and latitude from kilometers.

    Args:
        img_center_lon (float): Longitude of the image center.
        img_center_lat (float): Latitude of the image center. 
        img_span_km (float): Span of the image in kilometer.  

    Returns:
        tuple[float, float]: Span of the image in degrees longitude and latitude.
    """
    img_center = Point(img_center_lat, img_center_lon)
    img_upper = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 0)
    img_lower = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 180)
    img_left = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 90)
    img_right = geodesic(kilometers=(img_span_km / 2)).destination(img_center, 270)
    return (np.abs(img_left.longitude - img_right.longitude), np.abs(img_upper.latitude - img_lower.latitude))

In [8]:
print(compute_img_span(img_center_lon, img_center_lat, img_span_km))

(np.float64(0.04483696580646779), np.float64(0.04502246319079539))


# Maximum image span in extended metadata
Using the function `compute_img_span_km`, the original rgb metadata from the WILDS version of FMoW was extended to hold the image span for each sample in km.



In [9]:
import os
import pathlib

import pandas as pd
from geopy.distance import geodesic
from geopy.point import Point

file_path = os.path.abspath('')
PROJECT_ROOT = pathlib.Path(file_path).parent.parent.resolve()
DATA_DIR = PROJECT_ROOT / "data" / "fmow_landsat"
EXTENSION_FACTOR = 6.0

if not (os.path.exists(PROJECT_ROOT) and os.path.exists(DATA_DIR)):
    raise NotADirectoryError()

In [10]:
metadata = pd.read_csv(
    DATA_DIR / "rgb_metadata_extended.csv")

selected_splits = {"train", "val", "test"}
is_train_val_test = metadata["split"].isin(selected_splits)
metadata_selected = metadata.loc[is_train_val_test]

max_span = metadata_selected["img_span_km"].max()
download_span = max_span * EXTENSION_FACTOR 

print(f"The biggest image span in the dataset is {max_span:.2f} km.")
print(f"Six times this span results in {download_span:.2f} km as the desired context span for the extended dataset.")

The biggest image span in the dataset is 4.98 km.
Six times this span results in 29.90 km as the desired context span for the extended dataset.


In [11]:
download_span

np.float64(29.896581425173256)